<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Objective-and-safety-functions" data-toc-modified-id="Objective-and-safety-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Objective and safety functions</a></span></li><li><span><a href="#Define-search-space" data-toc-modified-id="Define-search-space-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define search space</a></span></li><li><span><a href="#Plot-objective-and-safety-functions" data-toc-modified-id="Plot-objective-and-safety-functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot objective and safety functions</a></span></li><li><span><a href="#Build-Gaussian-Process-models" data-toc-modified-id="Build-Gaussian-Process-models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Build Gaussian Process models</a></span></li><li><span><a href="#M-SafeOpt-(for-finding-global-safe-optimum)" data-toc-modified-id="M-SafeOpt-(for-finding-global-safe-optimum)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>M-SafeOpt (for finding global safe optimum)</a></span></li><li><span><a href="#M-SafeOpt-(for-finding-optimal-safe-action-for-every-x)" data-toc-modified-id="M-SafeOpt-(for-finding-optimal-safe-action-for-every-x)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>M-SafeOpt (for finding optimal safe action for every x)</a></span></li><li><span><a href="#SafeOpt-MC-Algorithm" data-toc-modified-id="SafeOpt-MC-Algorithm-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>SafeOpt-MC Algorithm</a></span></li><li><span><a href="#PredVar-Algorithm" data-toc-modified-id="PredVar-Algorithm-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>PredVar Algorithm</a></span></li><li><span><a href="#Set-Acquisition-Rule" data-toc-modified-id="Set-Acquisition-Rule-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Set Acquisition Rule</a></span></li><li><span><a href="#Run-Safe-Bayesian-Optimization" data-toc-modified-id="Run-Safe-Bayesian-Optimization-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Run Safe Bayesian Optimization</a></span></li><li><span><a href="#Plot-results" data-toc-modified-id="Plot-results-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Plot results</a></span></li><li><span><a href="#Pre-compute-values-for-regret-plots" data-toc-modified-id="Pre-compute-values-for-regret-plots-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Pre-compute values for regret plots</a></span></li><li><span><a href="#Cumulative-regret-plot-(finding-global-safe-optimum):-$R_t/t$" data-toc-modified-id="Cumulative-regret-plot-(finding-global-safe-optimum):-$R_t/t$-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Cumulative regret plot (finding global safe optimum): $R_t/t$</a></span></li><li><span><a href="#Cumulative-regret-plot-(finding-safe-optimum-for-every-$x$):-$R'_t/t$" data-toc-modified-id="Cumulative-regret-plot-(finding-safe-optimum-for-every-$x$):-$R'_t/t$-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Cumulative regret plot (finding safe optimum for every $x$): $R'_t/t$</a></span></li><li><span><a href="#Cumulative-regret-plot-(finding-safe-optimum-for-every-$x$):-$R^{\mathcal{X}}_t/t$" data-toc-modified-id="Cumulative-regret-plot-(finding-safe-optimum-for-every-$x$):-$R^{\mathcal{X}}_t/t$-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Cumulative regret plot (finding safe optimum for every $x$): $R^{\mathcal{X}}_t/t$</a></span></li></ul></div>

In [ ]:
!pip install trieste

In [ ]:
!pip install matplotlib

In [ ]:
!pip install plotly

In [ ]:
import os
import math
import gpflow
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from trieste.experimental.plotting import (
    plot_bo_points,
    plot_function_2d,
)

import trieste
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.data import Dataset
from trieste.models import TrainableModelStack
from trieste.models.gpflow import build_gpr, GaussianProcessRegression
from trieste.space import Box, SearchSpace


### Objective and safety functions

In [ ]:
from trieste.objectives.utils import mk_observer
from trieste.types import TensorType
from trieste.space import DiscreteSearchSpace
import random

from trieste.objectives import ScaledBranin
scaled_branin = ScaledBranin.objective

def obj_scaled_branin(x: TensorType) -> TensorType:
    obj_tensor = scaled_branin(x)
    obj_tensor = tf.squeeze(obj_tensor, -1)
    return obj_tensor

def g_syn_1(x: TensorType) -> TensorType:
    x0 = x[..., :1]
    y = x[..., 1:] + 1/3.0
    safety_tensor = -2*(x0)*( tf.exp(y)*tf.sin(10*y) + tf.sin(5*y) + 5)/3
    return tf.squeeze(safety_tensor, -1)

def dose_efficacy(x: TensorType) -> TensorType:
    x0 = x[..., :1]
    x1 = x[..., 1:]
    obj_tensor_1 = -1/(1+tf.exp(1 - 2*x0 - x1 + 4*x0**2 + x1**2))
    return tf.squeeze(obj_tensor_1, -1)

def dose_toxicity(x: TensorType) -> TensorType:
    x0 = x[..., :1]
    x1 = x[..., 1:]
    obj_tensor_1 = -1/(1+tf.exp(-2*x0-1*x1))
    return tf.squeeze(obj_tensor_1, -1)

# Change to set different functions as objective/safety functions
obj = dose_efficacy
safety = dose_toxicity

# Safety threshold
h = 0.9  

### Define search space

In [ ]:
def comb_funcs(x: TensorType) -> TensorType:
    return tf.stack([obj(x), safety(x)], axis = -1)

def comb_funcs_plot(x:TensorType) -> TensorType:
    return -comb_funcs(x)

observer = mk_observer(
    comb_funcs
)

mins = [0, 0]
maxs = [2, 1]

data_dim = 2
data_num_per_dim = tf.constant(200)

coords_per_axis = tf.linspace(tf.constant(mins, dtype=tf.float64), tf.constant(maxs, dtype=tf.float64), data_num_per_dim, axis=-1)
idx = list(range(1,coords_per_axis.shape[0]+1))
idx[-1] = 0
coords = tf.meshgrid(*tf.unstack(tf.gather(coords_per_axis, idx)))
coords = [tf.reshape(xv, [-1]) for xv in coords]
coords = tf.stack(coords, axis=-1)

search_space = DiscreteSearchSpace(coords)

num_objective = 2

### Plot objective and safety functions

In [ ]:
_, ax = plot_function_2d(
    comb_funcs_plot,
    search_space.lower,
    search_space.upper,
    contour=True,
    title=["Objective function", "Safety function"],
    figsize=(12, 5),
    colorbar=True,
    xlabel="$s$",
    ylabel="$x$",
)

safety_true = -1*safety(coords).numpy().reshape([data_num_per_dim**(data_dim-1), data_num_per_dim])
safety_true[safety_true > h] = -np.inf
safe_s_max_true = np.argmax(safety_true, axis = -1)

safe_boundary_true = np.hstack( (np.arange(data_num_per_dim**(data_dim-1)).reshape([data_num_per_dim**(data_dim-1), 1]), 
                            safe_s_max_true.reshape([data_num_per_dim**(data_dim-1), 1])) )
boundary_points_true = [coords[x[0] * data_num_per_dim + x[1] ] for x in safe_boundary_true]

ax[0, 1].plot([a[0] for a in boundary_points_true], [a[1] for a in boundary_points_true], color = "red", marker = '+', markevery = 15)


### Build Gaussian Process models

[Change GP parameters here, if necessary]

In [ ]:
import tensorflow_probability as tfp

def build_model(data, variance):
    variance = tf.math.reduce_variance(data.observations)
    kernel = gpflow.kernels.Matern52(variance=variance, lengthscales=[.2, .2])
    prior_scale = tf.cast(1.0, dtype=tf.float64)
    kernel.variance.prior = tfp.distributions.LogNormal(
        tf.cast(variance, dtype=tf.float64), prior_scale
    )
    
    kernel.lengthscales.prior = tfp.distributions.LogNormal(
        tf.math.log(kernel.lengthscales), prior_scale
    )
    
    gpr = gpflow.models.GPR(data.astuple(), kernel, noise_variance=1e-5)
    gpflow.set_trainable(gpr.likelihood, False)
    return GaussianProcessRegression(gpr, num_kernel_samples=10)

def build_stacked_independent_objectives_model(
    data: Dataset, num_output: int, search_space: SearchSpace, variances = [1,1]
) -> TrainableModelStack:
    gprs = []
    for idx in range(num_output):
        single_obj_data = Dataset(
            data.query_points, tf.gather(data.observations, [idx], axis=1)
        )
        gprs.append((build_model(single_obj_data, variances[idx]), 1))

    return TrainableModelStack(*gprs)

### M-SafeOpt (for finding global safe optimum)

In [ ]:
h = tf.constant(0.9, tf.float64)   #safety threshold

beta_f = tf.constant(3.0, tf.float64)   #scaling factor for confidence interval of GP_f
beta_g = tf.constant(3.0, tf.float64)   #scaling factor for confidence interval of GP_g

from trieste.acquisition import (
    SingleModelAcquisitionBuilder,
    ExpectedImprovement,
    Product,
)

class MSafeOpt(SingleModelAcquisitionBuilder):
    def prepare_acquisition_function(self, model, dataset=None):
        def acquisition(x):
            global L_f
            global l_g
            
            print("|",end=" ")
            mean, var = model.predict(tf.squeeze(x, -2))
            stddev = tf.math.sqrt(var)
                        
            mean_f = tf.reshape(mean[:,0], [mean.shape[0], 1])
            mean_g = tf.reshape(mean[:,1], [mean.shape[0], 1])
                                
            stddev_f = tf.reshape(stddev[:,0], [mean.shape[0], 1])
            stddev_g = tf.reshape(stddev[:,1], [mean.shape[0], 1])
                                  
            dims = tf.constant(x.shape[-1], mean.dtype)
            num_per_dim = tf.math.pow(tf.cast(x.shape[0], tf.float64), 1. / dims)  # number of discrete points per dimension
            num_per_dim = tf.cast(tf.math.round(num_per_dim), tf.int64)
            dims = tf.cast(dims, tf.int64)
            
            UCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) + beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            UCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) + beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            LCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) - beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            LCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) - beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            
            mask_less_g = tf.reshape(tf.math.less_equal(UCB_g, h), [mean.shape[0]])  # mask for all points with UCB <= h
            mask_more_g = tf.reshape(tf.math.greater(UCB_g, h), [mean.shape[0], 1])  # mask for all points with UCB > h
                        
            # if no points have UCB > h, set s^{(x)}_t=1 for all x
            if tf.math.count_nonzero(mask_more_g) == 0: 
                a = tf.zeros([num_per_dim ** (dims - 1), num_per_dim - 1], mean.dtype)
                b = tf.ones([num_per_dim ** (dims - 1), 1], mean.dtype)
                mask_combined = tf.concat([a, b], axis=-1)
                return tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                        tf.reshape(mask_combined, [stddev.shape[0], 1]))
            
            # mask for all points with s = 0
            mask_zero_g = tf.reshape(tf.math.equal(tf.reshape(tf.squeeze(x, -2)[:, 0], 
                                                            [mean.shape[0], 1]), tf.constant(0, mean.dtype)), [mean.shape[0], 1])  
                                  
            mask_zero_g = tf.reshape(tf.math.logical_and(mask_more_g, mask_zero_g),
                               [mean.shape[0], 1])  # mask for all points with s=0 and UCB > h
                                  
            mask_safe_g = tf.reshape(mask_less_g,  [mean.shape[0], 1])
            mask_safe_g = tf.math.logical_or(mask_safe_g, mask_zero_g)

            mask_less_g = tf.reshape(mask_less_g, [mean.shape[0], 1])

            mask_combined_g = tf.reshape(tf.math.logical_or(mask_less_g, mask_zero_g), [mean.shape[0], 1])
            mask_combined_g = tf.reshape(mask_combined_g, [num_per_dim ** (dims - 1), num_per_dim])
            true_indices_g = tf.math.argmax(tf.reverse(mask_combined_g, [-1]), axis=-1)
            const_g = (num_per_dim - 1) * tf.ones(num_per_dim ** (dims - 1), tf.int64)
            true_indices_g = const_g - true_indices_g
            mask_final_g = tf.zeros([num_per_dim ** (dims - 1), num_per_dim])
            mask_final_g = tf.tensor_scatter_nd_add(mask_final_g,
                                                  tf.stack([tf.range(num_per_dim ** (dims - 1)), true_indices_g], axis=1),
                                                  tf.ones(num_per_dim ** (dims - 1)))
            
            safe_UCB_f = tf.math.multiply(tf.cast(mask_safe_g, tf.float64), UCB_f + tf.constant(1000, mean.dtype))
            safe_UCB_f = tf.reshape(safe_UCB_f, [num_per_dim ** (dims - 1), num_per_dim])
            max_values = tf.reduce_max(safe_UCB_f, axis = -1, keepdims=True)
            mask_max_f = tf.greater_equal(safe_UCB_f, max_values)
            max_values = max_values - tf.constant(1000, mean.dtype)

            mask_final_g = tf.reshape(mask_final_g, [mean.shape[0], 1])
            
            # Eliminate suboptimal x's, and Limit expansion if suboptimal
            input_s = tf.reshape(tf.squeeze(x, -2)[:, 0], [mean.shape[0], 1])
            input_s_boundary = tf.math.multiply(input_s, tf.cast(mask_final_g, mean.dtype))
            input_s_boundary = tf.reshape(input_s_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            input_s_boundary = tf.reduce_max(input_s_boundary, axis = -1, keepdims=True)
            
            input_s_boundary_tiled = tf.tile(input_s_boundary,  [1 , num_per_dim])
            input_s_diff = tf.reshape(input_s, [num_per_dim ** (dims - 1), num_per_dim]) - input_s_boundary_tiled
            
            LCB_g_boundary = tf.math.multiply(LCB_g + tf.constant(1000, mean.dtype), tf.cast(mask_final_g, mean.dtype))
            LCB_g_boundary = tf.reshape(LCB_g_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            LCB_g_boundary = tf.reduce_max(LCB_g_boundary, axis = -1 , keepdims=True) - tf.constant(1000, mean.dtype)
            LCB_g_boundary = tf.tile(LCB_g_boundary,  [1,num_per_dim])
            pessimistic_LCB_g = LCB_g_boundary + l_g*input_s_diff
            pessimistic_mask_g = tf.less_equal(pessimistic_LCB_g, h)
            
            UCB_f_boundary =  tf.math.multiply(UCB_f + tf.constant(1000, mean.dtype), tf.cast(mask_final_g, mean.dtype))
            UCB_f_boundary = tf.reshape(UCB_f_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            UCB_f_boundary = tf.reduce_max(UCB_f_boundary, axis = -1, keepdims=True) - tf.constant(1000, mean.dtype)
            UCB_f_boundary = tf.tile(UCB_f_boundary,  [1,num_per_dim])
            optimistic_UCB_f = UCB_f_boundary + L_f*input_s_diff
            optimistic_UCB_f_masked = tf.math.multiply(optimistic_UCB_f+ tf.constant(1000, mean.dtype), tf.cast(pessimistic_mask_g, mean.dtype))
            optimistic_UCB_f_max = tf.reduce_max(optimistic_UCB_f_masked, axis = -1, keepdims=True)-tf.constant(1000, mean.dtype)
            
            LCB_f_safe =  tf.math.multiply(LCB_f+tf.constant(1000, mean.dtype), tf.cast(mask_safe_g, mean.dtype))
            LCB_f_safe = tf.reshape(LCB_f_safe, [num_per_dim ** (dims - 1), num_per_dim])
            LCB_f_safe_max = tf.reduce_max(LCB_f_safe) - tf.constant(1000, mean.dtype)
            
            expd_mask = tf.greater(optimistic_UCB_f_max, LCB_f_safe_max)
            expd_mask = tf.tile(expd_mask,[1, num_per_dim])
            expd_mask = tf.reshape(expd_mask, [mean.shape[0], 1])
                        
            elim_mask = tf.greater(max_values, LCB_f_safe_max)
            elim_mask = tf.tile(elim_mask, [1, num_per_dim])
            elim_mask = tf.reshape(elim_mask, [mean.shape[0], 1])

            elim_mask = tf.math.logical_or(elim_mask, expd_mask)       
            
            mask_final_g = tf.math.logical_and(tf.cast(mask_final_g, tf.bool), elim_mask)
            mask_final_g = tf.math.logical_and(tf.cast(mask_final_g, tf.bool), expd_mask)        

            mask_max_f = tf.reshape(mask_max_f, [mean.shape[0], 1])
            mask_max_f = tf.math.logical_and(mask_max_f, elim_mask)
            mask_final_f = tf.math.logical_or(tf.cast(mask_max_f, tf.bool), tf.cast(mask_final_g, tf.bool))
            
            
            max_interval_f = beta_f * tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                    tf.cast(mask_final_f, mean.dtype))
            
            max_interval_g = beta_g * tf.math.multiply(tf.reshape(stddev_g, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_final_g, [stddev.shape[0], 1]), mean.dtype))
            
            return tf.maximum(max_interval_f, max_interval_g)
                                          
        return acquisition
    
    

### M-SafeOpt (for finding optimal safe action for every x)

In [ ]:
class MSafeOptX(SingleModelAcquisitionBuilder):
    def prepare_acquisition_function(self, model, dataset=None):
        def acquisition(x):
            print("|",end=" ")
            global L_f
            global l_g
            
            mean, var = model.predict(tf.squeeze(x, -2))
            stddev = tf.math.sqrt(var)
                        
            mean_f = tf.reshape(mean[:,0], [mean.shape[0], 1])
            mean_g = tf.reshape(mean[:,1], [mean.shape[0], 1])
                                
            stddev_f = tf.reshape(stddev[:,0], [mean.shape[0], 1])
            stddev_g = tf.reshape(stddev[:,1], [mean.shape[0], 1])
                                  
            dims = tf.constant(x.shape[-1], mean.dtype)
            num_per_dim = tf.math.pow(tf.cast(x.shape[0], tf.float64), 1. / dims)  # number of discrete points per dimension
            num_per_dim = tf.cast(tf.math.round(num_per_dim), tf.int64)
            dims = tf.cast(dims, tf.int64)
            
            UCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) + beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            UCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) + beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            LCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) - beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            LCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) - beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            
            mask_less_g = tf.reshape(tf.math.less_equal(UCB_g, h), [mean.shape[0]])  # mask for all points with UCB <= h
            mask_more_g = tf.reshape(tf.math.greater(UCB_g, h), [mean.shape[0], 1])  # mask for all points with UCB > h
                        
            # if no points have UCB > h, set s^{(x)}_t=1 for all x
            if tf.math.count_nonzero(mask_more_g) == 0: 
                a = tf.zeros([num_per_dim ** (dims - 1), num_per_dim - 1], mean.dtype)
                b = tf.ones([num_per_dim ** (dims - 1), 1], mean.dtype)
                mask_combined = tf.concat([a, b], axis=-1)
                return tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                        tf.reshape(mask_combined, [stddev.shape[0], 1]))
            
            # mask for all points with s = 0
            mask_zero_g = tf.reshape(tf.math.equal(tf.reshape(tf.squeeze(x, -2)[:, 0], 
                                                            [mean.shape[0], 1]), tf.constant(0, mean.dtype)), [mean.shape[0], 1])  
                                  
            mask_zero_g = tf.reshape(tf.math.logical_and(mask_more_g, mask_zero_g),
                               [mean.shape[0], 1])  # mask for all points with s=0 and UCB > h
                                  
            mask_safe_g = tf.reshape(mask_less_g,  [mean.shape[0], 1])
            mask_safe_g = tf.math.logical_or(mask_safe_g, mask_zero_g)
            
            mask_less_g = tf.reshape(mask_less_g, [mean.shape[0], 1])

            mask_combined_g = tf.reshape(tf.math.logical_or(mask_less_g, mask_zero_g), [mean.shape[0], 1])
            mask_combined_g = tf.reshape(mask_combined_g, [num_per_dim ** (dims - 1), num_per_dim])
            true_indices_g = tf.math.argmax(tf.reverse(mask_combined_g, [-1]), axis=-1)
            const_g = (num_per_dim - 1) * tf.ones(num_per_dim ** (dims - 1), tf.int64)
            true_indices_g = const_g - true_indices_g
            mask_final_g = tf.zeros([num_per_dim ** (dims - 1), num_per_dim])
            mask_final_g = tf.tensor_scatter_nd_add(mask_final_g,
                                                  tf.stack([tf.range(num_per_dim ** (dims - 1)), true_indices_g], axis=1),
                                                  tf.ones(num_per_dim ** (dims - 1)))

            safe_UCB_f = tf.math.multiply(tf.cast(mask_safe_g, tf.float64), UCB_f)
            safe_UCB_f = tf.reshape(safe_UCB_f, [num_per_dim ** (dims - 1), num_per_dim])
            max_values = tf.reduce_max(safe_UCB_f, axis = -1, keepdims=True)
            mask_max_f = tf.greater_equal(safe_UCB_f, max_values)
            
            mask_final_g = tf.reshape(mask_final_g, [mean.shape[0], 1])
            
            # Limit expansion if suboptimal
            input_s = tf.reshape(tf.squeeze(x, -2)[:, 0], [mean.shape[0], 1])
            input_s_boundary = tf.math.multiply(input_s, tf.cast(mask_final_g, mean.dtype))
            input_s_boundary = tf.reshape(input_s_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            input_s_boundary = tf.reduce_max(input_s_boundary, axis = -1, keepdims=True)
            
            input_s_boundary_tiled = tf.tile(input_s_boundary,  [1 , num_per_dim])
            input_s_diff = tf.reshape(input_s, [num_per_dim ** (dims - 1), num_per_dim]) - input_s_boundary_tiled
            
            
            LCB_g_boundary = tf.math.multiply(LCB_g+tf.constant(1000, mean.dtype), tf.cast(mask_final_g, mean.dtype))
            LCB_g_boundary = tf.reshape(LCB_g_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            LCB_g_boundary = tf.reduce_max(LCB_g_boundary, axis = -1 , keepdims=True) - tf.constant(1000, mean.dtype)
            LCB_g_boundary = tf.tile(LCB_g_boundary,  [1,num_per_dim])
            pessimistic_LCB_g = LCB_g_boundary + l_g*input_s_diff
            pessimistic_mask_g = tf.less_equal(pessimistic_LCB_g, h)
            
            UCB_f_boundary =  tf.math.multiply(UCB_f, tf.cast(mask_final_g, mean.dtype))
            UCB_f_boundary = tf.reshape(UCB_f_boundary, [num_per_dim ** (dims - 1), num_per_dim])
            UCB_f_boundary = tf.reduce_max(UCB_f_boundary, axis = -1, keepdims=True)
            UCB_f_boundary = tf.tile(UCB_f_boundary,  [1,num_per_dim])
            optimistic_UCB_f = UCB_f_boundary + L_f*input_s_diff
            optimistic_UCB_f_masked = tf.math.multiply(optimistic_UCB_f, tf.cast(pessimistic_mask_g, mean.dtype))
            optimistic_UCB_f_max = tf.reduce_max(optimistic_UCB_f_masked, axis = -1, keepdims=True)
            
            LCB_f_safe =  tf.math.multiply(LCB_f+tf.constant(1000, mean.dtype), tf.cast(mask_safe_g, mean.dtype))
            LCB_f_safe = tf.reshape(LCB_f_safe, [num_per_dim ** (dims - 1), num_per_dim])
            LCB_f_safe_max = tf.reduce_max(LCB_f_safe, axis = -1, keepdims=True)  - tf.constant(1000, mean.dtype)  
            
            expd_mask = tf.greater(optimistic_UCB_f_max, LCB_f_safe_max)
            expd_mask = tf.tile(expd_mask, [1,num_per_dim])
            expd_mask = tf.reshape(expd_mask, [mean.shape[0], 1])
            
            mask_final_g = tf.math.logical_and(expd_mask, tf.cast(mask_final_g, tf.bool))
            
            mask_max_f = tf.reshape(mask_max_f, [mean.shape[0], 1])
            mask_final_f = tf.math.logical_or(tf.cast(mask_max_f, tf.bool), tf.cast(mask_final_g, tf.bool))
            
            max_interval_f = beta_f * tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_final_f, [stddev.shape[0], 1]), mean.dtype))
            max_interval_g = beta_g * tf.math.multiply(tf.reshape(stddev_g, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_final_g, [stddev.shape[0], 1]), mean.dtype))
            
            return tf.maximum(max_interval_f, max_interval_g)
                                          
        return acquisition
    

### SafeOpt-MC Algorithm


[Modified as per the description in our paper]

In [ ]:
class SafeOpt(SingleModelAcquisitionBuilder):
    def prepare_acquisition_function(self, model, dataset=None):
        def acquisition(x):
            print("|",end=" ")
            mean, var = model.predict(tf.squeeze(x, -2))
            stddev = tf.math.sqrt(var)
                        
            mean_f = tf.reshape(mean[:,0], [mean.shape[0], 1])
            mean_g = tf.reshape(mean[:,1], [mean.shape[0], 1])
                                
            stddev_f = tf.reshape(stddev[:,0], [mean.shape[0], 1])
            stddev_g = tf.reshape(stddev[:,1], [mean.shape[0], 1])
                                  
            dims = tf.constant(x.shape[-1], mean.dtype)
            num_per_dim = tf.math.pow(tf.cast(x.shape[0], tf.float64), 1. / dims)  # number of discrete points per dimension
            num_per_dim = tf.cast(tf.math.round(num_per_dim), tf.int64)
            dims = tf.cast(dims, tf.int64)
            
            UCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) + beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            UCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) + beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            LCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) - beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            LCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) - beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # lower confidence bound
            
            mask_less_g = tf.reshape(tf.math.less_equal(UCB_g, h), [mean.shape[0]])  # mask for all points with UCB <= h
            mask_more_g = tf.reshape(tf.math.greater(UCB_g, h), [mean.shape[0], 1])  # mask for all points with UCB > h
                                    
            # mask for all points with s = 0
            mask_zero_g = tf.reshape(tf.math.equal(tf.reshape(tf.squeeze(x, -2)[:, 0], 
                                                            [mean.shape[0], 1]), tf.constant(0, mean.dtype)), [mean.shape[0], 1])  
                                  
            mask_zero_g = tf.reshape(tf.math.logical_and(mask_more_g, mask_zero_g),
                               [mean.shape[0], 1])  # mask for all points with s=0 and UCB > h
                                  
            mask_safe_g = tf.reshape(mask_less_g,  [mean.shape[0], 1])
            mask_safe_g = tf.math.logical_or(mask_safe_g, mask_zero_g)

            mask_less_g = tf.reshape(mask_less_g, [mean.shape[0], 1])

            mask_combined_g = tf.reshape(tf.math.logical_or(mask_less_g, mask_zero_g), [mean.shape[0], 1])
            mask_combined_g = tf.reshape(mask_combined_g, [num_per_dim ** (dims - 1), num_per_dim])
            true_indices_g = tf.math.argmax(tf.reverse(mask_combined_g, [-1]), axis=-1)
            const_g = (num_per_dim - 1) * tf.ones(num_per_dim ** (dims - 1), tf.int64)
            true_indices_g = const_g - true_indices_g
            mask_final_g = tf.zeros([num_per_dim ** (dims - 1), num_per_dim])
            mask_final_g = tf.tensor_scatter_nd_add(mask_final_g,
                                                  tf.stack([tf.range(num_per_dim ** (dims - 1)), true_indices_g], axis=1),
                                                  tf.ones(num_per_dim ** (dims - 1)))
                                  
            correction_for_more_g = tf.cast(tf.cast(tf.math.count_nonzero(tf.reshape(mask_more_g, 
                                                [num_per_dim ** (dims - 1), num_per_dim]), -1), tf.bool), tf.int64)
            correction_for_more_g = tf.tile(tf.reshape(correction_for_more_g, [num_per_dim ** (dims - 1), 1]), 
                                            [1, num_per_dim])
            mask_final_g = tf.math.multiply(tf.cast(mask_final_g, tf.float64), tf.cast(correction_for_more_g, tf.float64))
            
            safe_UCB_f = tf.math.multiply(tf.cast(mask_safe_g, tf.float64), UCB_f)
            safe_UCB_f = tf.reshape(safe_UCB_f, [num_per_dim ** (dims - 1), num_per_dim])
            
            mask_final_g = tf.reshape(mask_final_g, [mean.shape[0], 1])
                        
            LCB_f_safe =  tf.math.multiply(LCB_f + tf.constant(1000, mean.dtype), tf.cast(mask_safe_g, mean.dtype))
            LCB_f_safe_max = tf.reduce_max(LCB_f_safe) - tf.constant(1000, mean.dtype)
            
            mask_max_f = tf.greater(safe_UCB_f, LCB_f_safe_max)
            mask_max_f = tf.reshape(mask_max_f, [mean.shape[0], 1])

            mask_final = tf.math.logical_or(tf.cast(mask_max_f, tf.bool), tf.cast(mask_final_g, tf.bool))
            
            max_interval_f = beta_f * tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                    tf.cast(mask_final, mean.dtype))
            
            max_interval_g = beta_g * tf.math.multiply(tf.reshape(stddev_g, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_final, [stddev.shape[0], 1]), mean.dtype))
            
            return tf.maximum(max_interval_f, max_interval_g)
                                          
        return acquisition
    

### PredVar Algorithm

In [ ]:
class PredVar(SingleModelAcquisitionBuilder):
    def prepare_acquisition_function(self, model, dataset=None):
        def acquisition(x):
            
            print("|",end=" ")
            mean, var = model.predict(tf.squeeze(x, -2))
            stddev = tf.math.sqrt(var)
            
            mean_f = tf.reshape(mean[:,0], [mean.shape[0], 1])
            mean_g = tf.reshape(mean[:,1], [mean.shape[0], 1])
                                
            stddev_f = tf.reshape(stddev[:,0], [mean.shape[0], 1])
            stddev_g = tf.reshape(stddev[:,1], [mean.shape[0], 1])
                                  
            dims = tf.constant(x.shape[-1], mean.dtype)
            num_per_dim = tf.math.pow(tf.cast(x.shape[0], tf.float64), 1. / dims)  # number of discrete points per dimension
            num_per_dim = tf.cast(tf.math.round(num_per_dim), tf.int64)
            dims = tf.cast(dims, tf.int64)

            
            UCB_f = - tf.reshape(mean_f, [mean.shape[0], 1]) + beta_f * tf.reshape(stddev_f,
                                                                         [stddev.shape[0], 1])  # upper confidence bound
            UCB_g = - tf.reshape(mean_g, [mean.shape[0], 1]) + beta_g * tf.reshape(stddev_g,
                                                                         [stddev.shape[0], 1])  # upper confidence bound

            mask_less_g = tf.reshape(tf.math.less_equal(UCB_g, h), [mean.shape[0], 1])  # mask for all points with UCB <= h
            mask_more_g = tf.reshape(tf.math.greater(UCB_g, h), [mean.shape[0], 1])  # mask for all points with UCB > h
            
            # mask for all points with s = 0
            mask_zero_g = tf.reshape(tf.math.equal(tf.reshape(tf.squeeze(x, -2)[:, 0], 
                                                            [mean.shape[0], 1]), tf.constant(0, mean.dtype)), [mean.shape[0], 1])  
                                  
            mask_zero_g = tf.reshape(tf.math.logical_and(mask_more_g, mask_zero_g),
                               [mean.shape[0], 1])  # mask for all points with s=0 and UCB > h
            
            mask_combined = tf.reshape(tf.math.logical_or(mask_less_g, mask_zero_g), [mean.shape[0], 1])
            
            stddev_f = tf.math.multiply(tf.reshape(stddev_f, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_combined, [stddev.shape[0], 1]), mean.dtype))
            stddev_g = tf.math.multiply(tf.reshape(stddev_g, [stddev.shape[0], 1]),
                                    tf.cast(tf.reshape(mask_combined, [stddev.shape[0], 1]), mean.dtype))
            
            return tf.maximum(stddev_f, stddev_g)

        return acquisition

### Set Acquisition Rule

In [ ]:
L_f = tf.constant(0.436, tf.float64)  # dose efficacy
l_g = tf.constant(0.035, tf.float64)  # dose toxicity

# L_f = tf.constant(10.166, tf.float64)  #scaled_branin
# l_g = tf.constant(0.86, tf.float64)  #g_syn_1

acq = MSafeOpt()
# acq = MSafeOptX()
# acq = SafeOpt()
# acq = PredVar()

rule: EfficientGlobalOptimization = EfficientGlobalOptimization(builder=acq)

### Run Safe Bayesian Optimization

In [ ]:
from trieste.ask_tell_optimization import AskTellOptimizer

num_runs = 1
num_steps = 50

for num_run in range(1, num_runs+1):

    model = build_stacked_independent_objectives_model(
        initial_data, num_objective, search_space
    )

    num_initial_points = 2
    initial_query_points = tf.constant([[0.0, maxs[0]*random.random()], [0.0, maxs[0]*random.random()]], dtype=tf.float64)
    initial_data = observer(initial_query_points)
    
    ask_tell = AskTellOptimizer(search_space, initial_data, model, acquisition_rule = rule)
    pred_means = []
    pred_vars = []    
    dataset = initial_data

    for step in range(1, num_steps + 1):
        print(f"Run #{num_run}: Step #{step}")
        new_point = ask_tell.ask()
        new_data_point = observer(new_point)
        dataset = dataset + new_data_point
        ask_tell.tell(dataset)       
        
        curr_mean, curr_var = model.predict(coords)
        pred_means.append(curr_mean.numpy())
        pred_vars.append(curr_var.numpy())

model_f = model._models[0].model
model_g = model._models[1].model

### Plot results

In [ ]:
def UCB_func(x):
    means, variances = model_f.predict_f(x)
    means = means.numpy()
    std_devs = np.sqrt(variances.numpy())  
    UCB_f = - means + beta_f.numpy()*(std_devs)
    
    means, variances = model_g.predict_f(x)
    means = means.numpy()
    std_devs = np.sqrt(variances.numpy())  
    UCB_g = - means + beta_g.numpy()*(std_devs)    

    return (UCB_f, UCB_g)
    
data_num_per_dim_plot = tf.constant(300)
coords_per_axis = tf.linspace(tf.constant(mins, dtype=tf.float64), tf.constant(maxs, dtype=tf.float64), data_num_per_dim, axis=-1)
idx = list(range(1,coords_per_axis.shape[0]+1))
idx[-1] = 0
coords_plot = tf.meshgrid(*tf.unstack(tf.gather(coords_per_axis, idx)))
coords_plot = [tf.reshape(xv, [-1]) for xv in coords_plot]
coords_plot = tf.stack(coords_plot, axis=-1)

UCB_f, UCB_g = UCB_func(coords_plot)
UCB_g = np.reshape(UCB_g, [data_num_per_dim**(data_dim-1), data_num_per_dim])
UCB_g[UCB_g > h] = -np.inf
safe_s_max = np.argmax(UCB_g, axis = -1)
safe_boundary = np.hstack( (np.arange(data_num_per_dim**(data_dim-1)).reshape([data_num_per_dim**(data_dim-1), 1]), 
                            safe_s_max.reshape([data_num_per_dim**(data_dim-1), 1])) )
boundary_points = [coords_plot[x[0] * data_num_per_dim + x[1] ] for x in safe_boundary]

safety_true = -1*safety(coords_plot).numpy().reshape([data_num_per_dim**(data_dim-1), data_num_per_dim])
safety_true[safety_true > h] = -np.inf
safe_s_max_true = np.argmax(safety_true, axis = -1)

safe_boundary_true = np.hstack( (np.arange(data_num_per_dim**(data_dim-1)).reshape([data_num_per_dim**(data_dim-1), 1]), 
                            safe_s_max_true.reshape([data_num_per_dim**(data_dim-1), 1])) )
boundary_points_true = [coords_plot[x[0] * data_num_per_dim + x[1] ] for x in safe_boundary_true]


In [ ]:
query_points = dataset.query_points.numpy()[2:]
observations = dataset.observations.numpy()[2:]

_, ax = plot_function_2d(
    comb_funcs_plot,
    search_space.lower,
    search_space.upper,
    contour=True,
    figsize=(12, 5),
    title=["Objective function", "Safety function"],
    xlabel="$s$",
    ylabel="$x$",
    colorbar=True,
)

plot_bo_points(data_query_points, ax=ax[0, 0], num_init=0)
plot_bo_points(data_query_points, ax=ax[0, 1], num_init=0)

ax[0, 1].plot([a[0] for a in boundary_points], [a[1] for a in boundary_points], color = "blue", marker = 'x', markevery = 9)
ax[0, 1].plot([a[0] for a in boundary_points_true], [a[1] for a in boundary_points_true], color = "red", marker = '+', markevery = 15)

### Pre-compute values for regret plots

In [ ]:
h = 0.9   # safety threshold

obj_values = -1*obj(coords).numpy()
safety_values = -1*safety(coords).numpy()

safe_obj_values = np.copy(obj_values)
safe_obj_values[safety_values > h] = -np.inf

max_obj_values = np.max(np.reshape(safe_obj_values, [data_num_per_dim**(data_dim-1), data_num_per_dim]), axis = -1)
max_obj_value = np.max(max_obj_values)

### Cumulative regret plot (finding global safe optimum): $R_t/t$

In [ ]:
def plot_cumulative_regret(
    obs_values,
    ax,
    label,
    obs_every = 2,
    show_obs=False,
    c = "blue",
):
    cum_values = np.cumsum(obs_values)
    cum_values_t = [cum_values[i]/(i+1) for i in range(len(obs_values))]
    ax.plot(cum_values_t, color=c, label=label + " (Avg. Cumulative)")
        
    if show_obs:
        obs_values = np.array(obs_values)
        ax.scatter(range(0,obs_values.shape[0], obs_every), obs_values[0::obs_every], c=c, marker='x', label=label+" (Instantaneous)")

suboptimality = []
obs_obj_values = observations[:, 0]
for i in range(obs_obj_values.shape[0]-1):
    curr_suboptimality = max_obj_value + obs_obj_values[i]
    suboptimality += [curr_suboptimality]
    
_, ax = plt.subplots(1, 1)

plot_cumulative_regret(
    suboptimality, ax, label = "Regret", show_obs = True
)

ax.legend(loc='upper right')
ax.set_xlabel("t",fontsize = 13)
ax.set_ylabel(r"$R_t/t$",fontsize = 13)

### Cumulative regret plot (finding safe optimum for every $x$): $R'_t/t$

In [ ]:
def get_x_index(query_point):
    return np.where(coords_per_axis.numpy()[0] == query_point[1])
    
suboptimality = []
obs_obj_values = observations[:, 0]

for i in range(obs_obj_values.shape[0]-1):
    x_index = get_x_index(query_points[i])
    curr_suboptimality = max_obj_values[x_index] + obs_obj_values[i]
    suboptimality += [curr_suboptimality]
    
_, ax = plt.subplots(1, 1)

plot_cumulative_regret(
    suboptimality, ax, label = "Regret", show_obs = True
)

ax.legend(loc='upper right')
ax.set_xlabel("t",fontsize = 13)
ax.set_ylabel(r"$R'_t/t$",fontsize = 13)

### Cumulative regret plot (finding safe optimum for every $x$): $R^{\mathcal{X}}_t/t$

In [ ]:
suboptimality_max = []
obj_values_reshaped = np.reshape(obj_values, [data_num_per_dim**(data_dim-1), data_num_per_dim])

for i in range(1, len(pred_means)):
    UCB_f = -pred_means[i][:,0] + beta_f.numpy()*pred_vars[i][:,0]
    UCB_g = -pred_means[i][:,1] + beta_g.numpy()*pred_vars[i][:,1]
    safe_UCB_f = UCB_f
    safe_UCB_f[UCB_g > h] = -np.inf

    max_UCB_f_index = np.argmax(np.reshape(safe_UCB_f, [data_num_per_dim**(data_dim-1), data_num_per_dim]), axis = -1)
    obj_values_best = []
    
    for j in range(data_num_per_dim**(data_dim-1)):
        obj_values_best.append(obj_values_reshaped[j, max_UCB_f_index[j]])

    max_UCB_regret = max_obj_values - np.array(obj_values_best)
    suboptimality_max.append(np.max(max_UCB_regret))

_, ax = plt.subplots(1, 1)

plot_cumulative_regret_x(
    suboptimality_max, ax, label = "Regret", show_obs = True
)

ax.legend(loc='upper right')
ax.set_xlabel("t",fontsize = 13)
ax.set_ylabel(r"$R^{\mathcal{X}}_t/t$",fontsize = 13)